# Como preparar e transformar dados usando Python

**Vamos mostrar nessa aula o que fazer ao receber um banco de dados _sujo_**:
- A remoção de linhas que contenham o dado;  
- A alteração dos dados específicos utilizando um número médio que não afete a análise.

**Também vamos aprender como Normalizar e Padronizar os dados**:
- Normalização e Padronização de dados
<a name="intro"></a>
### Sumário
1. [Removendo dados estranhos](#git1)
2. [Substituindo valores](#git2)
3. [Normalização e Padronização de dados](#git3)
    1. [Normalização](#git3.1)
    2. [Padronização](#git3.2)
4. [Binarização dos dados](#git4)
5. [Transformando variáveis nominais em números inteiros](#git5)
6. [One-hot encoding: transformando valores em códigos binários](#git6)
7. [Correlacionando dados](#git7)
8. [Dados desbalanceados: como detectar, analisar e balancear](#git8)
9. [Análise de Componentes Principais (PCA)](#git9)
10. [Boxplot: detectando, exibindo e descartando _outliers_](#git10)

Ao receber um conjunto de dados, provavelmente podem ser encontrados erros, inconsistências e informações dobradas. Antes de realizar qualquer análise estatística ou aplicação de algoritmos, devemos realizar uma limpeza na base de dados colhida.

Para a leitura e manipulação de tabelas, utilizaremos a biblioteca _pandas_, que pode ser facilmente instalada através do comando _pip install pandas_.

O banco de dados será o arquivo CSV (que pode ser aberto também no Excel e em outros leitores de planilha).

Também utilizaremos a biblioteca matplotlib, uma das mais utilizadas para visualizar os dados por meio de gráficos.
Vamos importar as bibliotecas!

In [166]:
import random
random.seed(1)

import pandas as pd
import matplotlib.pyplot as plt

### 1. Removendo dados estranhos <a name="git1"></a> [🠡](#intro)

Agora, vamos associar os dados "iris-with-errors.csv" à variável _data_, chamar o _print_ contendo o número de linhas e colunas através do comando shape e exibir as 25 primeiras linhas (incluindo o cabeçalho)

In [167]:
data = pd.read_csv('dados/iris-with-errors.csv', header=(0))
print("Linha, coluna:", data.shape)
data.head(25)

Linha, coluna: (25, 5)


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,duplicada
1,5.1,3.5,1.4,0.2,duplicada
2,?,3,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,5.1,3.5,1.4,0.2,duplicada
5,NaN,3.1,1.5,0.2,setosa
6,5,3.6,1.4,0.2,setosa
7,5.4,3.9,1.7,0.4,duplicada
8,5.4,3.9,1.7,0.4,duplicada
9,4.6,3.4,1.4,NaN,setosa


Podemos ver alguns erros:
1. O caractere "?"
2. A expressão _NaN_, quando o computador não sabe de que tipo é a informação
3. Na coluna _species_, são indicadas se as linhas são duplicadas

Precisamos rever os dados ou descartá-los. Aqui, vamos escolher a segunda opção e utilizar o pandas para isso.

In [168]:
data = data.dropna() # Remove os dados NaN
data.duplicated() # Verifica se há dados duplicados

0     False
1      True
2     False
3     False
4      True
6     False
7     False
8      True
10    False
11    False
12    False
13    False
14     True
15    False
16    False
17     True
18    False
19    False
20    False
21    False
22    False
23    False
dtype: bool

Podemos perceber no _Output_ acima que as linhas duplicadas receberam o estado **True**. De duas linhas iguais, somente a que vem depois da primeira é a "duplicada". Vamos descartar essas linhas!

In [169]:
data = data.drop_duplicates()
data.head(25)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,duplicada
2,?,3,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
6,5,3.6,1.4,0.2,setosa
7,5.4,3.9,1.7,0.4,duplicada
10,5,3.4,1.5,0.2,setosa
11,4.4,2.9,1.4,0.2,duplicada
12,4.9,3.1,1.5,0.1,setosa
13,5.4,3.7,1.5,0.2,setosa
15,4.8,3.4,1.6,0.2,setosa


Agora só falta removermos as interrogações "?". Um dos modos de fazer isso é transformar tal caractere em um _NaN_, e logo em seguida limpar novamente as linhas que contenham _NaN_.

In [170]:
import numpy as np
data = data.replace('?', np.nan) # Usa o comando replace para substituir as interrogações em NaN's
data = data.dropna() # Remove as linhas que contenham NaN
data.head(25)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,duplicada
3,4.7,3.2,1.3,0.2,setosa
6,5,3.6,1.4,0.2,setosa
7,5.4,3.9,1.7,0.4,duplicada
10,5,3.4,1.5,0.2,setosa
11,4.4,2.9,1.4,0.2,duplicada
12,4.9,3.1,1.5,0.1,setosa
13,5.4,3.7,1.5,0.2,setosa
15,4.8,3.4,1.6,0.2,setosa
16,4.8,3,1.4,0.1,setosa


Com os dados limpos, não precisamos mais das classificações da coluna **species**, certo? Vamos remove-la!

In [171]:
print("Vamos remover a coluna:", data.columns[4]) # Para exibir as colunas que serão removidas, use o comando .columns
data = data.drop(data.columns[4], axis=1) # Nesse comando, precisamos indicar o axis=1 que significa coluna
data.head(2) # Pra verificarmos se a coluna foi removida, podemos visualizar apenas poucas linhas.

Vamos remover a coluna: species


,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
3,4.7,3.2,1.3,0.2


Caso mais algum dado distoe e você queira remover diretamente uma linha escolhida, podemos utilizar o comando acima com a indicação **axis=0** indicando o número e o tipo linha.

In [172]:
print("Vamos remover as linhas:", data.index[[0, 2]]) # Para exibir as linhas que serão removidas, use o comando .index
data = data.drop(data.index[[0, 2]], axis=0)
data.head(25)

Vamos remover as linhas: Int64Index([0, 6], dtype='int64')


,sepal_length,sepal_width,petal_length,petal_width
3,4.7,3.2,1.3,0.2
7,5.4,3.9,1.7,0.4
10,5,3.4,1.5,0.2
11,4.4,2.9,1.4,0.2
12,4.9,3.1,1.5,0.1
13,5.4,3.7,1.5,0.2
15,4.8,3.4,1.6,0.2
16,4.8,3,1.4,0.1
18,4.3,3,1.1,0.1
19,5.8,4,1.2,0.2


Pronto! Removemos as linhas _[0, 2]_, ou seja, a primeira e a terceira linha, de número **0** e **6**.

### 2. Substituindo valores <a name="git2"></a> [🠡](#intro)
 
Se forem constatados valores ausentes, podemos substitui-los facilmente. Vamos utilizar agora a variável _data_ausente_, relacionada aos mesmos dados iniciais.

In [173]:
data_ausente = pd.read_csv('dados/iris-with-errors.csv', header=(0))
print(data_ausente.shape) # Número de linhas e colunas, lembra?
data_ausente.head(50) # Opa! Eu quero agora exibir 50 linhas, o que será que vai acontecer?

(25, 5)


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,duplicada
1,5.1,3.5,1.4,0.2,duplicada
2,?,3,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,5.1,3.5,1.4,0.2,duplicada
5,NaN,3.1,1.5,0.2,setosa
6,5,3.6,1.4,0.2,setosa
7,5.4,3.9,1.7,0.4,duplicada
8,5.4,3.9,1.7,0.4,duplicada
9,4.6,3.4,1.4,NaN,setosa


Agora nós estamos com dó de descartar os dados, pois a linha ainda possui informações usáveis. Um jeito inteligente é substituir os valores "?" e os "NaN", colocando no lugar um **valor médio**, ou seja, o valor que seria mais provável naquele lugar. Essa opção é boa para esse caso, pois não possuímos muitas linhas de dados, e removê-las ocasionaria em menos informação.

In [174]:
import numpy as np
data_ausente = data_ausente.replace('?', np.nan) # Transformamos os "?" em NaN


# Vamos usar o comando abaixo para transformar as linhas e colunas em formato Numpy (em Arrays)
X = np.array(data_ausente[data_ausente.columns[0:data_ausente.shape[1]-1]], dtype = float) # Também ignoramos a última coluna

averages = np.nanmean(X, axis = 0) # Usamos a função nanmean que calcula a média (ou mediana em alguns casos) ignorando os Nan
for i in np.arange(0, X.shape[0]):
    for j in np.arange(0, X.shape[1]):
        if(np.isnan(X[i,j]) == True): # Vamos verificar se é um dado NaN
            X[i,j] = averages[j] # Inserimos a média
print(X) # Exibimos o Array que foi construído, calculado e alterado

# Documentação - https://docs.scipy.org/doc/numpy/reference/generated/numpy.nanmean.html

[[5.1        3.5        1.4        0.2       ]
 [5.1        3.5        1.4        0.2       ]
 [5.02272727 3.         1.4        0.2       ]
 [4.7        3.2        1.3        0.2       ]
 [5.1        3.5        1.4        0.2       ]
 [5.02272727 3.1        1.5        0.2       ]
 [5.         3.6        1.4        0.2       ]
 [5.4        3.9        1.7        0.4       ]
 [5.4        3.9        1.7        0.4       ]
 [4.6        3.4        1.4        0.22608696]
 [5.         3.4        1.5        0.2       ]
 [4.4        2.9        1.4        0.2       ]
 [4.9        3.1        1.5        0.1       ]
 [5.4        3.7        1.5        0.2       ]
 [4.4        2.9        1.4        0.2       ]
 [4.8        3.4        1.6        0.2       ]
 [4.8        3.         1.4        0.1       ]
 [4.4        2.9        1.4        0.2       ]
 [4.3        3.         1.1        0.1       ]
 [5.8        4.         1.2        0.2       ]
 [5.7        4.4        1.5        0.4       ]
 [5.4        

### 3. Normalização e Padronização de dados <a name="git3"></a> [🠡](#intro)
#### 3.1 Normalização <a name="git3.1"></a> [🠡](#intro)
A **Normalização** é o método em que pegamos o maior dado da planilha e transformamos em **1**, e o menor em **0**. Os que estiverem entre eles serão **normalizados**. Para isso vamos utilizar a biblioteca sklearn que já faz isso para nós.

Importamos o banco de dados _iris.csv_ e associamos à variável _data_normalizada_, importamos as bibliotecas _numpy_ e _sklearn_. Transformamos a tabela em Array como da última vez, ignorando a última coluna.

**Atenção!!**: Não esqueça de instalar a nova biblioteca utilizando o comando _pip install sklearn_.

In [175]:
data_normalizada = pd.read_csv('dados/iris.csv', header=(0))

import numpy as np
from sklearn.preprocessing import MinMaxScaler

X = np.array(data_normalizada[data_normalizada.columns[0:data.shape[1]-1]]) # Transformação em Array ignorando última coluna
for i in range(X.shape[1]):
    print("MAIOR valor da coluna", i, "=", max(X[:,i]))
    print("MENOR Valor da coluna", i, "=", min(X[:,i]))
# prepara a função para transformar os dados
scaler = MinMaxScaler(feature_range=(0, 1)) # O mínimo e o máximo aqui será "0" e "1"
# Realiza a normalização e coloca em um novo vetor
X_norm = scaler.fit_transform(X) # A variável X_norm será a matriz criada através do comando .scaler
print(X_norm)

MAIOR valor da coluna 0 = 7.9
MENOR Valor da coluna 0 = 4.3
MAIOR valor da coluna 1 = 4.4
MENOR Valor da coluna 1 = 2.0
MAIOR valor da coluna 2 = 6.9
MENOR Valor da coluna 2 = 1.0
[[0.22222222 0.625      0.06779661]
 [0.16666667 0.41666667 0.06779661]
 [0.11111111 0.5        0.05084746]
 [0.08333333 0.45833333 0.08474576]
 [0.19444444 0.66666667 0.06779661]
 [0.30555556 0.79166667 0.11864407]
 [0.08333333 0.58333333 0.06779661]
 [0.19444444 0.58333333 0.08474576]
 [0.02777778 0.375      0.06779661]
 [0.16666667 0.45833333 0.08474576]
 [0.30555556 0.70833333 0.08474576]
 [0.13888889 0.58333333 0.10169492]
 [0.13888889 0.41666667 0.06779661]
 [0.         0.41666667 0.01694915]
 [0.41666667 0.83333333 0.03389831]
 [0.38888889 1.         0.08474576]
 [0.30555556 0.79166667 0.05084746]
 [0.22222222 0.625      0.06779661]
 [0.38888889 0.75       0.11864407]
 [0.22222222 0.75       0.08474576]
 [0.30555556 0.58333333 0.11864407]
 [0.22222222 0.70833333 0.08474576]
 [0.08333333 0.66666667 0.  

In [176]:
print('Mínimo dos atributos:', np.amin(X_norm, axis=0))
print('Máximo dos atributos:', np.amax(X_norm, axis=0))

Mínimo dos atributos: [0. 0. 0.]
Máximo dos atributos: [1. 1. 1.]


#### 3.2 Padronização <a name="git3.2"></a> [🠡](#intro)
A **Padronização** de dados possui o mesmo objetivo que a **Normalização**, que é o de transformar todos os dados para que fiquem em uma certa ordem de grandeza. A diferença é que na Padronização, **a média é igual a 0** e **o desvio padrão é igual a 1**.

Vamos utilizar a biblioteca **sklearn** para padronizar nossos dados!

In [177]:
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('dados/iris.csv', header=(0))
data_matriz = np.array(data[data.columns[0:data.shape[1]-1]]) # arquivo CSV transformado em matriz (array)
padronizador = StandardScaler().fit(data_matriz) # média = 0, desvio padrão = 1
matriz_padronizada = padronizador.transform(data_matriz)

print(matriz_padronizada)

[[-9.00681170e-01  1.03205722e+00 -1.34127240e+00 -1.31297673e+00]
 [-1.14301691e+00 -1.24957601e-01 -1.34127240e+00 -1.31297673e+00]
 [-1.38535265e+00  3.37848329e-01 -1.39813811e+00 -1.31297673e+00]
 [-1.50652052e+00  1.06445364e-01 -1.28440670e+00 -1.31297673e+00]
 [-1.02184904e+00  1.26346019e+00 -1.34127240e+00 -1.31297673e+00]
 [-5.37177559e-01  1.95766909e+00 -1.17067529e+00 -1.05003079e+00]
 [-1.50652052e+00  8.00654259e-01 -1.34127240e+00 -1.18150376e+00]
 [-1.02184904e+00  8.00654259e-01 -1.28440670e+00 -1.31297673e+00]
 [-1.74885626e+00 -3.56360566e-01 -1.34127240e+00 -1.31297673e+00]
 [-1.14301691e+00  1.06445364e-01 -1.28440670e+00 -1.44444970e+00]
 [-5.37177559e-01  1.49486315e+00 -1.28440670e+00 -1.31297673e+00]
 [-1.26418478e+00  8.00654259e-01 -1.22754100e+00 -1.31297673e+00]
 [-1.26418478e+00 -1.24957601e-01 -1.34127240e+00 -1.44444970e+00]
 [-1.87002413e+00 -1.24957601e-01 -1.51186952e+00 -1.44444970e+00]
 [-5.25060772e-02  2.18907205e+00 -1.45500381e+00 -1.31297673e

É interessante notar que agora possuímos valores **negativos** em nossa matriz padronizada. Isso acontece pois **a média é igual a zero**, além do desvio padrão igual a **um**.

Vamos calcular a média de cada coluna da nossa matriz padronizada!

In [178]:
for i in np.arange(0,matriz_padronizada.shape[1]):
    print('A Média da coluna', i, 'é:', np.mean(matriz_padronizada[:,i]))
    print('O desvio padrão da coluna', i, 'é:', np.std(matriz_padronizada[:,i]), '\n')

A Média da coluna 0 é: -4.736951571734001e-16
O desvio padrão da coluna 0 é: 1.0 

A Média da coluna 1 é: -6.631732200427602e-16
O desvio padrão da coluna 1 é: 0.9999999999999999 

A Média da coluna 2 é: 3.315866100213801e-16
O desvio padrão da coluna 2 é: 0.9999999999999998 

A Média da coluna 3 é: -2.842170943040401e-16
O desvio padrão da coluna 3 é: 1.0 



Podemos interpretar **e-16** como sendo **muito próximo de zero**

### 4. Binarização dos dados <a name="git4"></a> [🠡](#intro)
Utilizamos dados binários para casos em que certo valor = 0 e um outro valor = 1, normalmente indicando "não" e "sim", ou "desligado" e "ligado".

Ao receber dados de exame de sangue de diversas pessoas, nós gostaríamos de saber se essa pessoa está **deficiente de ferro**. Para isso, diremos que:
1. Deficiente de ferro se a ferritina está **abaixo** de 30 microgramas\L;
2. Não deficiente se a ferritina está **acima** de 30 microgramas\L.

Atente para o fato de que **não estar deficiente** de ferro NÃO significa que ela está normalizada, pois uma grande quantidade de ferro pode representar outro tipo de doença.

Sendo assim, os valores serão **binarizados**:
1. Deficiente de ferro será **0**;
2. Não deficiente de ferro será **1**.

Vamos **criar uma tabela fictícia** utilizando a biblioteca nativa **csv**. 

Depois vamos transformar os dados utilizando a biblioteca **sklearn** e suas etiquetas **Binarizer** e **MinMaxScaler**:

In [179]:
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import MinMaxScaler
import csv

#Criando uma tabela com dados fictícios
with open('dados/dados_ferritina_sangue.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    
    writer.writerow(["Código", "Ferritina em microgramas\L"])
    writer.writerow([32.8, "PACIENTE A"])       
    writer.writerow([30.1, "PACIENTE B"])
    writer.writerow([29.9, "PACIENTE C"])
    writer.writerow([27.6, "PACIENTE D"])
    writer.writerow([64.3, "PACIENTE E"])
    writer.writerow([45.1, "PACIENTE F"])

    
data_ferritina = pd.read_csv('dados/dados_ferritina_sangue.csv', header=(0), encoding = "ISO-8859-1")

# Transformando a tabela em matriz (array) e considerando apenas os atributos da coluna 1.
X = np.array(data_ferritina[data_ferritina.columns[0:1]])

T = 30 # valor do Limiar / Treshold
print('Limiar:', T)
print("-------------------")

# Binarização dos dados considerando o limiar T (threshhold)
binarizer = Binarizer(threshold=T).fit(X) #.fit(X_padronizado)
X_binarizado = binarizer.transform(X) #(X_padronizado)

for i in np.arange(0, X_padronizado.shape[0]):
    print("Antes:", X[i,])
    print("Depois:", X_binarizado[i, ])
    print("-------------------")

Limiar: 30
-------------------


NameError: name 'X_padronizado' is not defined

### 5. Transformando variáveis nominais em números inteiros <a name="git5"></a> [🠡](#intro)
Esse é mais simples. Caso queira transformar algum tipo de variável nominal em números inteiros, basta indicar **o termo a ser substituído** e **em que ele se transformará**:

In [ ]:
data = pd.read_csv('dados/iris.csv', header=(0))

classes = np.unique(data[data.columns[-1]])
number = 0 # valor que a classe será transformada

for i in classes:
    data = data.replace(i, number) # cada classe corresponderá a um valor, respectivamente
    number = number + 1 # esperamos que seja setosa = 0, versicolor = 1 e virginica = 2

classes_novas = np.unique(data[data.columns[-1]])

print("Dados antigos:", classes)
print("Novos dados:", classes_novas)

Também é possível fazer o processo contrário! Vamos pegar os dados de ferritina no sangue e, se o número for menor do que 30, ele será transformado em **DEFICIENTE EM FERRITINA**. Se for maior ou igual a 30, **NÃO DEFICIENTE EM FERRITINA**.

In [ ]:
data_ferritina = pd.read_csv('dados/dados_ferritina_sangue.csv', header=(0), encoding = "ISO-8859-1")
classes_ferritina = np.unique(data_ferritina[data_ferritina.columns[0]])

deficiente = "DEFICIENTE EM FERRITINA" # valor STRING em que a classe será transformada
nao_deficiente = "NÃO DEFICIENTE EM FERRITINA" # valor STRING em que a classe será transformada
number = 30.0

print(data_ferritina)

for i in classes_ferritina:
    if i < 30:
        data_ferritina = data_ferritina.replace(i, deficiente)
    else:
        data_ferritina = data_ferritina.replace(i, nao_deficiente)



classes_ferritina_novas = np.unique(data_ferritina[data_ferritina.columns[0]])

#print("\nDados antigos:", classes_ferritina,)
#print("Novos dados:", classes_ferritina_novas)

print("\nAgora a tabela ficou assim:\n")
print(data_ferritina)

### 6. One-hot encoding: transformando valores em códigos binários <a name="git6"></a> [🠡](#intro)
O **one-hot encoding** é uma outra forma de trabalhar com binários. Pode ser feito de maneira automática com poucas linhas de código. Basicamente, contará o número de variáveis diferentes, atribuindo valor binário com mais de uma coluna.

Com o exemplo, ficará muito mais compreensível:

In [ ]:
import pandas as pd

# Vamos criar um DataFrame com o pandas
df = pd.DataFrame ({'A':['a', 'b', 'c', 'd'],}) # Podemos contar QUATRO variáveis diferentes aqui: a, b, c, d
df.head()

In [ ]:
df = pd.get_dummies(df) # Vamos usar o one-hot encoding
df.head()

Viu como transformamos os diferentes atributos em **códigos binários**?
- O **a** agora é 1 0 0 0
- O **d** agora é 0 0 0 1

### 7. Correlacionando dados <a name="git7"></a> [🠡](#intro)
**Correlacionar dados** significa identificar, dentre variantes, colunas ou qualquer outro tipo de atributo, **quais os atributos que possuem maior ou menor correlação**, e medi-la.

Para exemplificar, vamos correlacionas os dados da base de dados **BostonHousing**, que é bem conhecida e que relaciona Casas com Preços 

In [ ]:
data_housing = pd.read_csv('dados/BostonHousing.csv', header=(0))
data_housing.head(5)

In [ ]:
corr = data_housing.corr()  # corr é o método de correlação do Pandas. Acabamos de correlacionar os dados!

# Daqui para baixo, estaremos gerando o gráfico de correlação
plt.figure(figsize=(7,7))
plt.imshow(corr, cmap='Blues', interpolation='none', aspect='auto') # imshow exibe data como uma imagem
plt.colorbar() # gera a barra do lado direito

# Vamos incluir o nome de todas as variáveis
plt.xticks(range(len(corr)), corr.columns, rotation='vertical') # adiciona as variáveis na linha X
plt.yticks(range(len(corr)), corr.columns); # adiciona as variáveis na linha Y
plt.suptitle('Correlation between variables', fontsize=15, fontweight='bold') # adiciona um título
plt.grid(False)
plt.show()

Analisar o gráfico pode não ser tão intuitivo assim. Se você ainda não sabe como fazer, é bem simples! A correlação está sendo contada de **0** a **1**, como mostra a barra lateral direita.

Nessa mesma barra lateral, podemos ver que:
- Quanto **mais clara** a cor, **menos correlação** entre variáveis x e y temos, ou seja, correlação se aproxima de **0**;
- Quanto **mais escura** a cor, **mais correlação** entre variáveis x e y temos, ou seja, correlação se aproxima de **1**.

Nós geramos um gráfico para que seja melhor de entender o que está ocorrendo nesse _dataset_, porém saiba que os dados estão armazenados em uma matriz:

In [ ]:
print(corr)

Percebeu que o **número máximo** é **1**? Isso está ocorrendo por conta do método utilizar a **Correlação de Pearson**. Aqui está a descrição do Wikipedia:

Em estatística descritiva, o coeficiente de correlação de Pearson, também chamado de "coeficiente de correlação produto-momento" ou simplesmente de "ρ de Pearson" mede o grau da correlação (e a direcção dessa correlação - se positiva ou negativa) entre duas variáveis de escala métrica (intervalar ou de rácio/razão).

Este coeficiente, normalmente representado por ρ assume apenas valores entre -1 e 1:
- **ρ = 1** significa uma **correlação positiva perfeita** entre as duas variáveis;
- **ρ = -1** significa uma **correlação negativa perfeita** entre as duas variáveis. **Se uma aumenta, a outra sempre diminui**;
- **ρ = 0** significa que as duas variáveis **não dependem linearmente uma da outra**. No entanto, pode existir uma dependência não linear. Assim, o resultado **ρ = 0** deve ser investigado por outros meios."


Vamos agora identificar quais as variáveis que possuem **maior correlação**:

In [ ]:
p = 0.75 # Essa é a correlação mínima que estamos considerando
var = []
for i in corr.columns: # percorre toda a tabela para cara elemento "i"
    for j in corr.columns: # percorre novamente toda a tabela para cada elemento "j"
        if(i != j):
            if np.abs(corr[i][j]) > p: # Se a correlação for maior que "p"
                var.append([i,j]) # Coloca na lista "var"
                
print('As variáveis com maior correlação:\n', var)

Esse tipo de método é muito importante pois também nos ajuda a filtrar melhor o _dataset_. Se dois dados são muito correlacionados, um deles pode ser removido, pois podem ter um significado muito próximo e interferir em outros dados.

### 8. Dados desbalanceados: como detectar, analisar e balancear <a name="git8"></a> [🠡](#intro)
Em uma amostragem de dados, possivelmente alguns deles podem estar **desbalanceados**. Isso ocorre quando o _dataset_ possui quantidades diferentes de elementos por classe. Quanto **maior a distância** entre a quantidade de elementos entre classes, **maior o desbalanceamento**. Vamos entender melhor com o próximo exemplo.

Para calcularmos quantas classes possui um _dataset_, basta contarmos através da coluna de classes. No _dataset_ abaixo, a coluna de classes é a última, que indica qual o tipo de veículo da linha correspondente.

In [ ]:
import pandas as pd

data_veiculo = pd.read_csv('dados/Vehicle.csv', header=(0))
data_veiculo.head(10)

In [ ]:
coluna_class = data_veiculo[data_veiculo.columns[-1]] # Vamos colocar a coluna "class" em uma variável
print(coluna_class)

Agora vamos separar a coluna class por elementos e contar a ocorrência de cada um deles.

In [ ]:
tipos_de_classe = np.unique(coluna_class) # Vamos pegar o nome dos diferentes elementos da coluna "class"
print(tipos_de_classe)
print(tipos_de_classe[2])

In [ ]:
# Vamos pegar o número de ocorrência de cada uma das quatro diferentes classes guardadas na variável acima
numero_de_classes = np.zeros(len(tipos_de_classe)) # pega qual a quantidade de classes diferentes, que é quatro


for i in np.arange(0, len(tipos_de_classe)): # para os elementos "i" de "0" até "quantidade de classes diferentes"
    a = coluna_class == tipos_de_classe[i] # "a" é variável da coluna_class, que é equivalente ao tipo de classe "i"
    numero_de_classes[i] = len(coluna_class[a]) # quantidade de aparição do elemento "i"
    
print(numero_de_classes)

Uma boa forma de analisar visualmente é utilizando o gráfico do tipo **Histograma**, que exibirá a quantidade de ocorrência por elemento (por tipo de veículo).

In [ ]:
import matplotlib.pyplot as plt

numbers = np.arange(0, len(tipos_de_classe))
plt.bar(numbers, numero_de_classes, alpha=.75)

# Agora vamos exibir o nome das classes ao invés de exibir números
plt.xticks(numbers, tipos_de_classe)
plt.title('Número de elementos por classe')
plt.show(True)

Ao obter a quantidade de elementos por classe, e ao analisar o Histograma acima, é possível concluir que **as classes estão bem distribuídas**, possuindo pouca distância entre quantidades. 

Porém, ainda assim podemos realizar uma redistribuição de dados para que tenhamos um **balanceamento perfeito**. O resultado será cada classe com o mesmo número de elementos.

In [ ]:
N = 3 # Vamos selecionar apenas três ocorrências de cada classe e seus respectivos atributos.

# classes
cl = np.unique(coluna_class) #pega o número de diferentes classes
X = np.array(data_veiculo) #transforma o dataset em uma matriz/array
Xnew = [] # este será o array que receberá appends, por enquanto está vazio
cls = np.array(data_veiculo[data_veiculo.columns[-1]])

for i in np.arange(0, len(cl)): # de 0 até o tamanho(número de diferentes classes)
    a = np.argwhere(cls == cl[i])
    inds = np.random.choice(a[:,0], N, replace=False) # Aleatoriamente, seleciona um elemento "a" qualquer "N" vezes
    Xnew.append(X[inds,:]) # append do valor armazenado na variável inds
    
Xnew = np.array(Xnew) # transforma Xnew em array estruturado

print('Dados obtidos a partir da amostragem')
print(Xnew)

### 9. Análise de Componentes Principais (PCA) <a name="git9"></a> [🠡](#intro)
A **Análise de Componentes Principais** ou **Principal Components Analisis (PCA)** envolve teoria de **espectro de matrizes**, o que foge da álgebra linear. Por fugir muito das análises vistas aqui nesse guia, a teoria não será passada POR ENQUANTO.

Ele é utilizado com a finalidade de analisar dados eliminando sobreposições e utilizando as formas mais representativas dos dados.

Vamos realizar essa análise através de poucas linhas de código! O mais interessante é o resultado gráfico.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv('dados/Iris.csv', header=(0))
coluna_class = np.unique(data[data.columns[-1]]) # Vamos guardar a última coluna, 'Class'
print(data.shape) # Através desse print, vemos que possui 150 linhas e 5 colunas
list_labels = list(data.columns)
data.head(10)

In [ ]:
data = data.to_numpy() # Transformando o dataset em formato NumPy
nrow,ncol = data.shape
y = data[:,-1] 
X = data[:,0:ncol-1] # '-1' para ignorar a última coluna, que é a de Classes

Vamos **padronizar** os dados utilizando o **sklearn**:

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

Agora vamos finalmente realizar a **PCA**:

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2) # Número de componentes
pca_resultado = pca.fit_transform(X)

# Vamos alterar os atributos do gráfico e exibi-lo utilizando o matplotlib
plt.figure(figsize=(8,5))
plt.scatter(pca_resultado[:,0], pca_resultado[:,1], s=50, color = 'blue') # primeiro '0' e segundo eixo '1'
plt.xlabel("Primeiro componente", fontsize=20)
plt.ylabel("Segundo componente", fontsize=20)
plt.xticks(color='k', size=20)
plt.yticks(color='k', size=20)
plt.show(True)

In [ ]:
# Agora vamos colorir de acordo com a classe dos dados, o que facilita muito na hora de analisar!
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']
aux = 0

plt.figure(figsize=(8,5))

for c in coluna_class:
    nodes = np.where(y == c)
    plt.scatter(pca_resultado[nodes,0], pca_resultado[nodes,1], s=50, color = colors[aux], label = c)
    aux = aux + 1
    
plt.legend()   
plt.xlabel("Primeiro componente", fontsize=20)
plt.ylabel("Segundo componente", fontsize=20)
plt.xticks(color='k', size=20)
plt.yticks(color='k', size=20)
plt.show(True)

É possível perceber como fica fácil, através da análise **PCA**, o quão próximas ou distantes estão as classes:
1. A setosa é muito diferente da versicolor e da virginica;
2. A versicolor e virginica são diferentes;
3. Entre a versicolor e a virginica, alguns valores se confundem.

### 10. Boxplot: detectando, exibindo e descartando _outliers_ <a name="git10"></a> [🠡](#intro)
Vamos verificar se há a presença de **outliers** em nossos _datasets_! Para isso, utilizaremos o gráfico do tipo **Boxplot**. 

Os **outliers** são dados que possuem diferenças significativas em relação à maioria. Eles podem prejudicar muito o cálculo da **Média** de um conjunto de dados.

Para gerar o tal gráfico **Boxplot**, são necessários alguns elementos, que serão calculados apenas na próxima aula!
1. A **Mediana** de um conjunto de dados;
1. **1º e 3º Quartil** de um conjunto de dados;
2. Um valor arbitrário de Mínimo e Máximo. Quem estiver fora dessa definição é considerado um **outlier**.

Vamos importar o _dataset_ "iris.csv" e exibir o Boxplot dele.

In [ ]:
import seaborn as sns # É uma biblioteca de exibição gráfica
import pandas as pd
import matplotlib.pyplot as plt

data_iris = pd.read_csv('dados/iris.csv', header=(0))
print(data_iris) # Vamos exibir o print para compararmos com o gráfico

plt.figure(figsize=(8, 8))

# Vamos alterar a exibição dos elementos do Boxplot e exibi-lo
sns.boxplot(x="species", y="petal_length", data=data_iris) # É aqui que definimos o que será o atributo "x" e o "y"
plt.xlabel('Espécie', fontsize=18)
plt.ylabel('Comprimento da pétala', fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show(True)

Perceba como conseguimos gerar o Boxplot da variante **petal_length** de cada uma das classes existentes em **species**. A linha do código que realizou essa definição foi essa:

``sns.boxplot(x="species", y="petal_length", data=data_iris) # É aqui que definimos o que será o atributo "x" e o "y"``

Na próxima aula, aprenderemos muito mais sobre como funcionam os cálculos. Aqui estamos apenas indicando uma forma fácil de encontrar esses dados e exibi-los graficamente.

Vamos agora detectar a distância interquantil (Q3 - Q1) para encontrar os **outliers** e exibilos graficamente!

In [ ]:
import pandas as pd

# Vamos gerar dados de forma randômica valores de '1' a '200' para as variáveis 'a', 'b' e 'c'
data = pd.DataFrame({'a': np.random.randint(1, 200, 20),
                    'b': np.random.randint(1, 200, 20),
                    'c': np.random.randint(1, 200, 20)})

# Vamos agora gerar alguns outliers. Os valores acima de 150 serão multiplicados em 10.
data[data > 150] = data[data > 150]*10
print(data)

Vamos exibir um gráfico do _dataset_ criado utilizando a biblioteca **seaborn**.

In [ ]:
import seaborn
seaborn.pairplot(data)
plt.show()

Agora vamos detectar os outliers utilizando a **distância interquartil** (**IQR**).

In [ ]:
Q1 = data.quantile(0.25) #primeiro quartil, ou seja 25% do dataset
Q3 = data.quantile(0.75) #terceiro quartil, ou seja, 75% do dataset
IQR = Q3 - Q1

print(((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR))))

True são os valores **menores que (Q1 - 1.5 * IQR) e maiores que (Q3 + 1.5 * IQR)**, ou seja, são os os elementos **outliers**, que ultrapassam os limites do gráfico **Boxplot**, visto anteriormente. O Boxplot **exclui** os dados outliers da análise, para não interferir na exibição e leitura gráfica.

Novamente, se restaram dúvidas, não se preocupe pois trataremos desse dados **outliers** na próxima aula.

Por fim, **vamos deletar** as linhas que contenham algum dado outlier, para que ele não interfira em outros tipos de análise:

In [ ]:
v = ((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR))).any(axis=1) # axis=1 significa linha, axis=0 significa coluna
data = data.drop(data.index[list(v)], axis=0) # Vamos deletar os valores 'v' utilizando data.drop
data.head(10)

Perceba como agora só ficaram as linhas em que todos os atributos tinham classificação **False** para outlier.

**Basicamente, essas são as formas mais utilizadas de exibição e tratamento de dados na Ciência de Dados.** Até o próximo guia!